In [1]:
import os
from config import *

In [2]:
for path in [TRAIN_DATA, VAL_DATA]:
	for i in os.listdir(path):
		if i in TRUE_NAME.keys():
			os.rename(
				os.path.join(path, i),
				os.path.join(path, TRUE_NAME[i].replace(' ', '_').lower())
			)
print(os.listdir(path))

['australian_terrier', 'beagle', 'border_terrier', 'dingo', 'english_foxhound', 'golden_retriever', 'old_english_sheepdog', 'rhodesian_ridgeback', 'samoyed', 'shih-tzu']


In [3]:
from torchvision.datasets import ImageFolder
from torchvision import transforms as T

from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim import Adam

import numpy as np

In [4]:
train_augmentation = T.Compose([
	T.RandomRotation((0, 90)),
	T.RandomVerticalFlip(p=.33),
	T.Resize([IMAGE_SIZE, IMAGE_SIZE]),
	T.ToTensor(),
	T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_augmentation = T.Compose([
	T.Resize(IMAGE_SIZE),
	T.ToTensor(),
	T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
# it is impossible change augmentations if ImageFolder split via random_split
train_data = ImageFolder(TRAIN_DATA, transform=train_augmentation)
valid_data = ImageFolder(TRAIN_DATA, transform=test_augmentation)

# obtain training indices
indices = np.arange(len(train_data))
np.random.shuffle(indices)
split = int(0.8 * len(train_data))
train_idx, valid_idx = indices[split:], indices[:split]

# prepare data loaders
train_loader = DataLoader(
	train_data, batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(train_idx), num_workers=12
)
valid_loader = DataLoader(
	train_data, batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(valid_idx), num_workers=12
)

In [6]:
from engine import *

In [7]:
lr = 1e-3

model = get_model().cuda()
model.train()
optimizer = Adam(model.parameters(), lr)
loss_function = nn.CrossEntropyLoss()

In [ ]:
plot_result(
	*train_classification(
		5, model, train_loader, valid_loader, loss_function, optimizer, noti_rate=150
	)
)

Train batch: 150 train loss: 2.3083, train accuracy: 0.6358
Train batch: 300 train loss: 2.3056, train accuracy: 0.6645


In [ ]:
torch.save(model, os.path.join(WEIGHT_PATH, "classification_5.model"))